In [45]:
from pathlib import Path
import pandas as pd

In [46]:
data_folder =  "/media/sz/Data/Connected_Lecturers/Opal/raw"
data_file_content = "OPAL_content.p"
data_file_similarity = "OPAL_similaritiy.p"
data_file_attrib = "OPAL_files_attrib.p"
data_file_ai = "OPAL_ai_meta.p"

In [47]:
df_content = pd.read_pickle(Path(data_folder) / data_file_content)
df_similarity = pd.read_pickle(Path(data_folder) / data_file_similarity)
df_attrib = pd.read_pickle(Path(data_folder) / data_file_attrib)
df_ai = pd.read_pickle(Path(data_folder) / data_file_ai)

Wie viele verschiedene Dokumente gibt es in unserem Datensatz überhaupt?

In [48]:
df_content["pipe:content_hash"].unique().shape[0]

5605

Wieviele sind schlicht und einfach mehrfach vorhanen?

In [49]:
df_content.shape[0] - df_content["pipe:content_hash"].unique().shape[0]

1757

Ok, was ist das am häufigsten vorkommende Dokument?

In [50]:
df_dublicates = df_content.groupby("pipe:content_hash").agg(
    occurance_count=('pipe:ID', 'count'), 
    first_file_ID=('pipe:ID', 'first'),
).sort_values("occurance_count", ascending=False).reset_index()

df_dublicates.drop(['pipe:content_hash'], axis=1, inplace=True)
df_dublicates.rename(columns={"first_file_ID": "pipe:ID"}, inplace=True)

# Merge df_dublicates with df_attrib to get the opal:oer_permalink
df_dublicates = df_dublicates.merge(df_attrib[["pipe:ID", "opal:oer_permalink"]], on="pipe:ID")

from IPython.display import HTML
HTML(df_dublicates.head(10).to_html(render_links=True, escape=False))

,occurance_count,pipe:ID,opal:oer_permalink
0,51,15kwF4Q161dAQ,https://bildungsportal.sachsen.de/opal/oer/15kwF4Q161dAQ
1,29,7pGAl0_xuWT0,https://bildungsportal.sachsen.de/opal/oer/7pGAl0_xuWT0
2,17,75-b_yi3C1Uk,https://bildungsportal.sachsen.de/opal/oer/75-b_yi3C1Uk
3,17,7i6iO956mG1U,https://bildungsportal.sachsen.de/opal/oer/7i6iO956mG1U
4,14,1Xjd0hs1bOxE,https://bildungsportal.sachsen.de/opal/oer/1Xjd0hs1bOxE
5,13,109a1MYOVrJkY,https://bildungsportal.sachsen.de/opal/oer/109a1MYOVrJkY
6,13,71MYNt2hZdq8,https://bildungsportal.sachsen.de/opal/oer/71MYNt2hZdq8
7,13,10JnVEe72gCW8,https://bildungsportal.sachsen.de/opal/oer/10JnVEe72gCW8
8,13,7jCaN9avlweU,https://bildungsportal.sachsen.de/opal/oer/7jCaN9avlweU
9,13,10pRfCF0klGfQ,https://bildungsportal.sachsen.de/opal/oer/10pRfCF0klGfQ


In [51]:
df_similarity[df_similarity["ai:filename"]=="15kwF4Q161dAQ"]

,ai:filename,ai:filename_similar,ai:similarity
3663,15kwF4Q161dAQ,7FmgyULxEBVw,1.0
3664,15kwF4Q161dAQ,3Iuw6QCg_WnA,1.0
3665,15kwF4Q161dAQ,15-6ldnxFmNK0,1.0


In [52]:
df_similarity[df_similarity["ai:filename"]=="7pGAl0_xuWT0"]

,ai:filename,ai:filename_similar,ai:similarity
8784,7pGAl0_xuWT0,81AFVYAcaVK0,1.0
8785,7pGAl0_xuWT0,8MQZqtNpdHzc,1.0
8786,7pGAl0_xuWT0,56kAAIIIvFRc,1.0


Ok, also lasst uns die Dokument rauswerfen die eine Kopie sind.

In [72]:
similarity_threshold = 0.9

df_match = df_similarity[(df_similarity["ai:similarity"]<0.9999999) & (df_similarity["ai:similarity"]>similarity_threshold)]
df_match.shape

(2488, 3)

## Vorbereitung des Plots

In [73]:
df_connections = df_match.merge(df_attrib[["pipe:ID", "opal:oer_permalink"]], right_on="pipe:ID", left_on="ai:filename").\
                    drop(["pipe:ID"], axis=1)
df_connections = df_connections.merge(df_attrib[["pipe:ID", "opal:oer_permalink"]], right_on="pipe:ID", left_on="ai:filename_similar").\
                    drop(["pipe:ID"], axis=1)

HTML(df_connections.head(10).to_html(render_links=True, escape=False))

,ai:filename,ai:filename_similar,ai:similarity,opal:oer_permalink_x,opal:oer_permalink_y
0,4-dHpo4qmwVg,10-sCOfJIf7ag,0.926229,https://bildungsportal.sachsen.de/opal/oer/4-dHpo4qmwVg,https://bildungsportal.sachsen.de/opal/oer/10-sCOfJIf7ag
1,4-dHpo4qmwVg,10c0Y1nWRRmU8,0.916175,https://bildungsportal.sachsen.de/opal/oer/4-dHpo4qmwVg,https://bildungsportal.sachsen.de/opal/oer/10c0Y1nWRRmU8
2,4-dHpo4qmwVg,4vv4Zb4dwl5U,0.912411,https://bildungsportal.sachsen.de/opal/oer/4-dHpo4qmwVg,https://bildungsportal.sachsen.de/opal/oer/4vv4Zb4dwl5U
3,1mjbqKfwSW7U,1UOraoEiVqec,0.928287,https://bildungsportal.sachsen.de/opal/oer/1mjbqKfwSW7U,https://bildungsportal.sachsen.de/opal/oer/1UOraoEiVqec
4,1Qhnsa15Gixs,1rCuHnmnp2g4,0.954104,https://bildungsportal.sachsen.de/opal/oer/1Qhnsa15Gixs,https://bildungsportal.sachsen.de/opal/oer/1rCuHnmnp2g4
5,1UOraoEiVqec,1mjbqKfwSW7U,0.928287,https://bildungsportal.sachsen.de/opal/oer/1UOraoEiVqec,https://bildungsportal.sachsen.de/opal/oer/1mjbqKfwSW7U
6,13AX_MPrEYic,1BruMQFjEIRY,0.969270,https://bildungsportal.sachsen.de/opal/oer/13AX_MPrEYic,https://bildungsportal.sachsen.de/opal/oer/1BruMQFjEIRY
7,13AX_MPrEYic,17n9nrGMxd_Ak,0.969270,https://bildungsportal.sachsen.de/opal/oer/13AX_MPrEYic,https://bildungsportal.sachsen.de/opal/oer/17n9nrGMxd_Ak
8,13AX_MPrEYic,10XJHRtjohoeI,0.969270,https://bildungsportal.sachsen.de/opal/oer/13AX_MPrEYic,https://bildungsportal.sachsen.de/opal/oer/10XJHRtjohoeI
9,1s-vtDJ00WzA,18r_JtdoG2RA,0.991046,https://bildungsportal.sachsen.de/opal/oer/1s-vtDJ00WzA,https://bildungsportal.sachsen.de/opal/oer/18r_JtdoG2RA


In [74]:
set_IDs = set(df_match["ai:filename"].values) | set(df_match["ai:filename_similar"].values)

df_net_attrib = df_attrib[df_attrib["pipe:ID"].isin(set_IDs)][["pipe:ID", "opal:oer_permalink"]]
df_net_attrib = df_net_attrib.merge(df_ai[["pipe:ID", "ai:author", "ai:keywords", "ai:dewey"]], on="pipe:ID")

def extract_major(dewey):
    if dewey == "":
        return np.nan
    try:
        return int(dewey[0])
    except:
        return np.nan

df_net_attrib['dewey_major']=df_net_attrib['ai:dewey'].apply(extract_major)

HTML(df_net_attrib.head(10).to_html(render_links=True, escape=False))

,pipe:ID,opal:oer_permalink,ai:author,ai:keywords,ai:dewey,dewey_major
0,4-dHpo4qmwVg,https://bildungsportal.sachsen.de/opal/oer/4-dHpo4qmwVg,,"Endodontie, Zahne, Behandlung, Überkappung, Prognose",617.6,6.0
1,1mjbqKfwSW7U,https://bildungsportal.sachsen.de/opal/oer/1mjbqKfwSW7U,TU Dresden,"E-Learning, TU Dresden, UrhWissG, Medienzentrum, §60a",,NaN
2,1BruMQFjEIRY,https://bildungsportal.sachsen.de/opal/oer/1BruMQFjEIRY,Medienzentrum,"OPAL, TU-Dresden, Login, Portlet, Favoriten",005,0.0
3,1Qhnsa15Gixs,https://bildungsportal.sachsen.de/opal/oer/1Qhnsa15Gixs,TU Dresden Medienzentrum,"Umfrage, Invote, E-Learning, TU Dresden, Sammlung",001,0.0
4,18r_JtdoG2RA,https://bildungsportal.sachsen.de/opal/oer/18r_JtdoG2RA,Medienzentrum,"OPAL, TU Dresden, E-Learning, Portlets, ZIH-Login",641,6.0
5,1UOraoEiVqec,https://bildungsportal.sachsen.de/opal/oer/1UOraoEiVqec,TU Dresden Medienzentrum,"Urheberrecht, E-Learning, Quelle, Lizenz, OER",346,3.0
6,13AX_MPrEYic,https://bildungsportal.sachsen.de/opal/oer/13AX_MPrEYic,TU Dresden Medienzentrum,"Kurs, Favoriten, Lehren, Lernen, OPAL",641,6.0
7,1s-vtDJ00WzA,https://bildungsportal.sachsen.de/opal/oer/1s-vtDJ00WzA,TU Dresden Medienzentrum,"OPAL, TU Dresden, ZIH-Login, Teach & Learn, Catalogue",005,0.0
8,9KYncn6He4vc,https://bildungsportal.sachsen.de/opal/oer/9KYncn6He4vc,Anna Hagedorn,"Tiere, Pflanzen, Wald, Kreuzworträtsel, Quiz",570,5.0
9,1fCUEOJoo3Hg,https://bildungsportal.sachsen.de/opal/oer/1fCUEOJoo3Hg,,"Gradient, Lokale Extrema, Gradientenvektorfeld, Graphenfläche, Extrempunkte",51,5.0


In [79]:
from pyvis.network import Network
import numpy as np
from matplotlib import cm
import matplotlib.colors as mcolors


def create_similarity_network(data, attributes=None,similarity_threshold=0.5):

    net = Network(height="750px", width="100%", bgcolor="#ffffff", font_color="black")
    all_files = set(data['ai:filename'].unique()) | set(data['ai:filename_similar'].unique())
    
    # Farbpalette aus dem gewählten Farbschema generieren
    colormap = cm.get_cmap("coolwarm")
    # Wir generieren 10 Farben (für Werte 0-9)
    color_palette = [mcolors.rgb2hex(colormap(i/9)) for i in range(10)]

    # Füge Knoten mit URLs hinzu
    for file in all_files:
        # Erstelle den HTML-Tooltip mit Link
        attachment = attributes[attributes['pipe:ID']==file]
        url = attachment['opal:oer_permalink'].values[0]
        author = attachment['ai:author'].values[0]
        dewey = attachment['ai:dewey'].values[0]
        title = f"""
            <div style='max-width:400px'>
                <b>Datei:</b> {file}<br>
                <b>Author:</b> {author}<br>
                <b>Dewey:</b> {dewey}<br>
                <b>Link:</b> <a href='{url}' target='_blank'>{url}</a>
            </div>
        """

        dewey_major = attachment['dewey_major'].values[0]
        if np.isnan(dewey_major):
            bg_color = '#000000'
        else:
            bg_color = color_palette[int(dewey_major)]            

        color={
             'background': bg_color,
        } 
        size=20              

        if author == "":
            bd_color ='#FF5722'
            color['border'] = bd_color
        else:
            color['border'] = bg_color

        net.add_node(
            file,
            label=file,
            title=title,
            customProperties={'url': url},
            size=17,
            borderWidth = 1,
            color=color
        )

    # Füge Kanten für Ähnlichkeiten über dem Schwellenwert hinzu
    for _, row in data.iterrows():
        if row['ai:similarity'] >= similarity_threshold:
            # Berechne Kantenbreite basierend auf Ähnlichkeit (1-5 Pixel)
            width = 1 + 4 * row['ai:similarity']
            
            color = '#000000'
            
            net.add_edge(
                row['ai:filename'],
                row['ai:filename_similar'],
                value=width,
                title=f"Ähnlichkeit: {row['ai:similarity']:.2f}",
                color=color
            )
    
    # Netzwerk-Optionen für bessere Darstellung
    #net.toggle_physics(True)
    #net.show_buttons(filter_=['physics'])
        
    return net

In [80]:
net = create_similarity_network(df_match, df_net_attrib, similarity_threshold=0.5)
net.show('similarity_network.html',  notebook=False)

/tmp/ipykernel_925674/2177971914.py:13: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colormap = cm.get_cmap("coolwarm")


similarity_network.html
